# Dogs breeds

https://youtu.be/JNxcznsrRb8?t=1h31m8s

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.torch_imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [3]:
torch.cuda.set_device(0)

Kaggle Dog Breed Identification. Get data from https://www.kaggle.com/c/dog-breed-identification

In [4]:
PATH = "G:/Other_Datasets/dogbreed/"
sz = 224
arch = resnext101_64
bs = 20

In [5]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1 # header is not counted (-1)
val_idxs = get_cv_idxs(n) # random 20% data for validation set

In [6]:
n

10222

In [7]:
len(val_idxs)

2044

In [ ]:
# If you haven't downloaded weights.tgz yet, download the file.
#     http://forums.fast.ai/t/error-when-trying-to-use-resnext50/7555
#     http://forums.fast.ai/t/lesson-2-in-class-discussion/7452/222
#!wget -O fastai/weights.tgz http://files.fast.ai/models/weights.tgz

#!tar xvfz fastai/weights.tgz -C fastai

## Initial exploration

In [ ]:
os.listdir(PATH)

In [ ]:
label_df = pd.read_csv(label_csv)

In [ ]:
label_df.head()

In [ ]:
label_df.pivot_table(index="breed", aggfunc=len).sort_values('id', ascending=False)

In [ ]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test', # we need to specify where the test set is if you want to submit to Kaggle competitions
                                   val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

In [ ]:
fn = PATH + data.trn_ds.fnames[0]; fn

In [ ]:
img = PIL.Image.open(fn); img

In [ ]:
img.size

In [ ]:
size_d = {k: PIL.Image.open(PATH + k).size for k in data.trn_ds.fnames}

In [ ]:
row_sz, col_sz = list(zip(*size_d.values()))

In [ ]:
row_sz = np.array(row_sz); col_sz = np.array(col_sz)

In [ ]:
row_sz[:5]

In [ ]:
plt.hist(row_sz);

In [ ]:
plt.hist(row_sz[row_sz < 1000])

In [ ]:
plt.hist(col_sz);

In [ ]:
plt.hist(col_sz[col_sz < 1000])

In [ ]:
len(data.trn_ds), len(data.test_ds)

In [ ]:
len(data.classes), data.classes[:5]

## Initial model

In [8]:
def get_data(sz, bs): # sz: image size, bs: batch size
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                       val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/13
    # http://forums.fast.ai/t/how-to-train-on-the-full-dataset-using-imageclassifierdata-from-csv/7761/37
    # 短边放大到 340
    return data if sz > 300 else data.resize(340, 'tmp') # Reading the jpgs and resizing is slow for big images, so resizing them all to 340 first saves time
    # 这里 val_idxs 表示验证集序列，其他都是训练集序列
#Source:   
#    def resize(self, targ, new_path):
#        new_ds = []
#        dls = [self.trn_dl,self.val_dl,self.fix_dl,self.aug_dl]
#        if self.test_dl: dls += [self.test_dl, self.test_aug_dl]
#        else: dls += [None,None]
#        t = tqdm_notebook(dls)
#        for dl in t: new_ds.append(self.resized(dl, targ, new_path))
#        t.close()
#        return self.__class__(new_ds[0].path, new_ds, self.bs, self.num_workers, self.classes)
#File:      ~/fastai/courses/dl1/fastai/dataset.py

### Precompute

In [18]:
len(data.val_ds)
len(data.trn_ds)

2044

In [12]:
data = get_data(sz, bs)

In [13]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [14]:
learn.fit(1e-2, 5)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.589209   0.355486   0.894814  
    1      0.478794   0.359006   0.893836                                                                              
    2      0.368333   0.333495   0.892857                                                                              
    3      0.275216   0.283411   0.909002                                                                              
    4      0.266213   0.291437   0.91047                                                                               



[0.2914373735211965, 0.9104696797997984]

### Augment

In [19]:
from sklearn import metrics

In [20]:
data = get_data(sz, bs)

In [21]:
learn = ConvLearner.pretrained(arch, data, precompute=True, ps=0.5)

In [22]:
learn.fit(1e-2, 2) # precompute=True 导致 data argument 失效

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.719004   0.357932   0.889433  
    1      0.48053    0.331552   0.898239                                                                              



[0.33155194568582635, 0.898238764117142]

In [23]:
learn.precompute = False

In [24]:
learn.fit(1e-2, 5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.440215   0.258487   0.918297  
    1      0.441676   0.260631   0.920254                                                                              
    2      0.370863   0.267276   0.917319                                                                              
    3      0.392728   0.25174    0.919276                                                                              
    4      0.320435   0.251285   0.918787                                                                              



[0.2512849209698714, 0.9187867043069664]

In [25]:
learn.save('224_pre')

In [26]:
learn.load('224_pre')

## Increase size

In [27]:
# Starting training on small images for a few epochs, then switching to bigger images, and continuing training is an amazingly effective way to avoid overfitting.

# http://forums.fast.ai/t/planet-classification-challenge/7824/96
# set_data doesn’t change the model at all. It just gives it new data to train with.
learn.set_data(get_data(299, bs)) 
learn.freeze()
# If you trained a model on smaller size images, 
# you can then call learn.set_data and pass in a larger size dataset. 
# That is going to take your model, however it has been trained so far, 
# and it is going to let you continue to train on larger images.
#Source:   
#    def set_data(self, data, precompute=False):
#        super().set_data(data)
#        if precompute:
#            self.unfreeze()
#            self.save_fc1()
#            self.freeze()
#            self.precompute = True
#        else:
#            self.freeze()
#File:      ~/fastai/courses/dl1/fastai/conv_learner.py

In [28]:
learn.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 299, 299]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(9408))])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('trainable', False),
                           ('nb_params', tensor(128))])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 150, 150]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 150, 150]),
                           ('output_shape', [-1, 64, 75, 75]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape

In [29]:
learn.fit(1e-2, 3, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.344134   0.241173   0.925636  
    1      0.360403   0.241336   0.920744                                                                              
    2      0.373959   0.245493   0.920254                                                                              



[0.24549343114536085, 0.9202544173615786]

Validation loss is much lower than training loss. This is a sign of underfitting. Cycle_len=1 may be too short. Let's set cycle_mult=2 to find better parameter.

In [30]:
# When you are under fitting, it means cycle_len=1 is too short (learning rate is getting reset before it had the chance to zoom in properly).
learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2) # 1+2+4 = 7 epochs

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.354936   0.239128   0.919765  
    1      0.313805   0.251171   0.922211                                                                              
    2      0.27944    0.237069   0.92319                                                                               
    3      0.341713   0.26784    0.920254                                                                              
    4      0.296738   0.240038   0.926125                                                                              
    5      0.207396   0.229554   0.932975                                                                              
    6      0.214003   0.22848    0.928571                                                                              



[0.228480278276426, 0.9285714407023152]

Training loss and validation loss are getting closer and smaller. We are on right track.

In [31]:
log_preds, y = learn.TTA() # (5, 2044, 120), (2044,)
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.9324853228962818, 0.2079982643590632)

In [32]:
len(data.val_ds.y), data.val_ds.y[:5]

(2044, array([19, 15,  7, 99, 73], dtype=int64))

In [33]:
learn.save('299_pre')

In [34]:
learn.load('299_pre')

In [35]:
learn.fit(1e-2, 1, cycle_len=2) # 1+1 = 2 epochs

epoch      trn_loss   val_loss   accuracy                                                                              
    0      0.244614   0.246779   0.925147  
    1      0.222621   0.229187   0.931018                                                                              



[0.2291872503502256, 0.9310176249469563]

In [36]:
learn.save('299_pre')

In [37]:
log_preds, y = learn.TTA()
probs = np.mean(np.exp(log_preds),0)
accuracy_np(probs, y), metrics.log_loss(y, probs)

(0.9339530332681018, 0.21613602096877077)

This dataset is so similar to ImageNet dataset. Training convolution layers doesn't help much. We are not going to unfreeze.

## Create submission

https://youtu.be/9C06ZPF8Uuc?t=1905

In [38]:
data.classes

['affenpinscher',
 'afghan_hound',
 'african_hunting_dog',
 'airedale',
 'american_staffordshire_terrier',
 'appenzeller',
 'australian_terrier',
 'basenji',
 'basset',
 'beagle',
 'bedlington_terrier',
 'bernese_mountain_dog',
 'black-and-tan_coonhound',
 'blenheim_spaniel',
 'bloodhound',
 'bluetick',
 'border_collie',
 'border_terrier',
 'borzoi',
 'boston_bull',
 'bouvier_des_flandres',
 'boxer',
 'brabancon_griffon',
 'briard',
 'brittany_spaniel',
 'bull_mastiff',
 'cairn',
 'cardigan',
 'chesapeake_bay_retriever',
 'chihuahua',
 'chow',
 'clumber',
 'cocker_spaniel',
 'collie',
 'curly-coated_retriever',
 'dandie_dinmont',
 'dhole',
 'dingo',
 'doberman',
 'english_foxhound',
 'english_setter',
 'english_springer',
 'entlebucher',
 'eskimo_dog',
 'flat-coated_retriever',
 'french_bulldog',
 'german_shepherd',
 'german_short-haired_pointer',
 'giant_schnauzer',
 'golden_retriever',
 'gordon_setter',
 'great_dane',
 'great_pyrenees',
 'greater_swiss_mountain_dog',
 'groenendael',


In [39]:
data.test_ds.fnames

['test\\000621fb3cbb32d8935728e48679680e.jpg',
 'test\\00102ee9d8eb90812350685311fe5890.jpg',
 'test\\0012a730dfa437f5f3613fb75efcd4ce.jpg',
 'test\\001510bc8570bbeee98c8d80c8a95ec1.jpg',
 'test\\001a5f3114548acdefa3d4da05474c2e.jpg',
 'test\\00225dcd3e4d2410dd53239f95c0352f.jpg',
 'test\\002c2a3117c2193b4d26400ce431eebd.jpg',
 'test\\002c58d413a521ae8d1a5daeb35fc803.jpg',
 'test\\002f80396f1e3db687c5932d7978b196.jpg',
 'test\\0036c6bcec6031be9e62a257b1c3c442.jpg',
 'test\\0041940322116ae58c38130f5a6f71f9.jpg',
 'test\\0042d6bf3e5f3700865886db32689436.jpg',
 'test\\004476c96f575879af4af471af65cae8.jpg',
 'test\\00485d47de966a9437ad3b33ac193b6f.jpg',
 'test\\00496f65de6cc319145ce97bd6e90360.jpg',
 'test\\004bf14426d1a830d459a9e0c0721309.jpg',
 'test\\004c3721eb88358f462cdcec6b2380b7.jpg',
 'test\\00559f56aab7e0a7749220f6aed65162.jpg',
 'test\\005b281f1a4d6f29d527c9585e9bd33c.jpg',
 'test\\005b6c6c76fefd6b458ef6fb6e54da6e.jpg',
 'test\\006870b49353779b25eeb91fed43c31a.jpg',
 'test\\0068f

In [ ]:
log_preds, y = learn.TTA(is_test=True) # use test dataset rather than validation dataset
probs = np.mean(np.exp(log_preds),0)
#accuracy_np(probs, y), metrcs.log_loss(y, probs) # This does not make sense since test dataset has no labels

In [ ]:
probs.shape # (n_images, n_classes)

In [ ]:
df = pd.DataFrame(probs)
df.columns = data.classes

In [ ]:
df.insert(0, 'id', [o[5:-4] for o in data.test_ds.fnames])

In [ ]:
df.head()

In [ ]:
SUBM = f'{PATH}/subm/'
os.makedirs(SUBM, exist_ok=True)
df.to_csv(f'{SUBM}subm.gz', compression='gzip', index=False)

In [ ]:
FileLink(f'{SUBM}subm.gz')

## Individual prediction

In [ ]:
fn = data.val_ds.fnames[0]
fn

In [ ]:
Image.open(PATH + fn).resize((150, 150))

In [ ]:
# Method 1.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
ds = FilesIndexArrayDataset([fn], np.array([0]), val_tfms, PATH)
dl = DataLoader(ds)
preds = learn.predict_dl(dl)
np.argmax(preds)

In [ ]:
learn.data.classes[np.argmax(preds)]

In [ ]:
# Method 2.
trn_tfms, val_tfms = tfms_from_model(arch, sz)
im = val_tfms(open_image(PATH + fn)) # open_image() returns numpy.ndarray
preds = learn.predict_array(im[None])
np.argmax(preds)